# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-nh' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-467-nh.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt-467-nh.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-nh/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-nh.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

> The result matched expectations and mirrored the results of a similar query from lab1.

**SQL Query**


```
SELECT DISTINCT
    Sub_Category
  FROM
    `mgmt-467-47888.lab1_foundation.superstore`
  WHERE Region = 'West'
ORDER BY
  Sub_Category
```



### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

<br/>

**Reflection:**
The top customer spend significantly more than the average customer with a distinct fall off in customert profit with each customer that follows before the drop off somewhat levels off at the 8th customer. Further investigation into customers 10-20 shows a much smaller decrease in profit from customer to customer.

**SQL Query:**
```
SELECT
    Customer_ID,
    SUM(Profit) AS total_profit
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
GROUP BY
    Customer_ID
ORDER BY
    total_profit DESC
LIMIT 10
```

### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

1. Total Count Verification: Sum the order_count column from the query result. This total should exactly match the result of running a simple COUNT(*) query on the source table filtered only by WHERE Category = 'Technology'.

2. Spot Check: Pick the top Ship_Mode (likely 'Standard Class') and the bottom Ship_Mode. Run a separate query filtering for Category = 'Technology' AND the specific Ship_Mode chosen, verifying that the resulting count matches the corresponding order_count in the main output.

<br/>

**Reflection:**
Standard class is by far the most common shipping method used, second class and first class are both used at about a 3rd of the frequency of standard, and same day is used very rarely.

**SQL Query:**
```
SELECT
    Ship_Mode,
    COUNT(Ship_Mode) AS order_count
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
WHERE
    Category = 'Technology'
GROUP BY
    Ship_Mode
ORDER BY
    order_count DESC
```

## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

**Reflection**
The dataset is old with the most recent sale coming from November 2017. Initially the model produced a query that looked at sales revenue from the past 12 months currently, and an adjustment was needed to find revenue for the past 12 most recent months in the dataset.

**SQL Query**


```
SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(Order_Date, MONTH)) AS year_month,
    SUM(Sales) AS monthly_revenue
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
WHERE
    -- 1. Define the end boundary: The first day of the month where the MAX Order_Date falls.
    -- This ensures we exclude the potentially partial last month.
    Order_Date < DATE_TRUNC((SELECT MAX(Order_Date) FROM `mgmt-467-nh.lab1_foundation.superstore_clean`), MONTH)
    
    -- 2. Define the start boundary: 12 months prior to the end boundary.
    AND Order_Date >= DATE_SUB(
        DATE_TRUNC((SELECT MAX(Order_Date) FROM `mgmt-467-nh.lab1_foundation.superstore_clean`), MONTH),
        INTERVAL 12 MONTH
    )
GROUP BY
    year_month
ORDER BY
    year_month ASC
```



### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.
>HAVING is necessary because it is used to filter groups based on the result of an aggregate function (SUM(Profit)), a comparison that the WHERE clause cannot perform as it operates on individual rows prior to aggregation.

<br/>

**Reflection:**
Tables, bookcases, and supplies are the sub categories with a negative profit.

**SQL Query:**


```
SELECT
    Sub_Category,
    SUM(Profit) AS total_profit
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
GROUP BY
    Sub_Category
HAVING
    SUM(Profit) < 0
ORDER BY
    total_profit ASC

```



## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

<br/>

**Reflection:**
Canon imageCLASS 2200 Advanced C has the most total sales.
**SQL Query**


```
WITH ProductDimension AS (
    -- Simulates the dimension table by getting distinct Product IDs and Names
    SELECT DISTINCT
        Product_ID,
        Product_Name
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
),
FactSales AS (
    -- Aggregates sales by Product ID
    SELECT
        Product_ID,
        SUM(Sales) AS total_sales
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
    GROUP BY
        Product_ID
)
SELECT
    d.Product_ID,
    d.Product_Name,
    f.total_sales
FROM
    ProductDimension AS d
INNER JOIN
    FactSales AS f
    ON d.Product_ID = f.Product_ID
ORDER BY
    total_sales DESC
```



## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

<br/>

**Reflection**
Texas, NY, Virginia, California are the top states in each region NESW respectivley.

**SQL Query**
```
WITH state_sales AS (
    -- Calculate total sales for every State within each Region
    SELECT
        Region,
        State,
        SUM(Sales) AS total_sales
    FROM
        `[YOUR_PROJECT].superstore_data.sales`
    GROUP BY
        Region,
        State
),
ranked_state_sales AS (
    -- Rank the states based on total_sales, partitioning the rank calculation by Region
    SELECT
        Region,
        State,
        total_sales,
        RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
    FROM
        state_sales
)
-- Select the top 3 states per Region
SELECT
    Region,
    State,
    total_sales,
    sales_rank
FROM
    ranked_state_sales
WHERE
    sales_rank <= 3
ORDER BY
    Region,
    sales_rank
```

### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

<br/>

**Refletion:**
The dataset only contains orders from as late as 2017, the query needed to be adjusted to look at revenue increase from 2016-2017.
**SQL Query**


```
WITH MaxYears AS (
    -- Dynamically find the latest year (Y2) and the year immediately preceding it (Y1)
    SELECT
        MAX(EXTRACT(YEAR FROM Order_Date)) AS Y2,
        MAX(EXTRACT(YEAR FROM Order_Date)) - 1 AS Y1
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
),
yr_sales AS (
    -- Calculate total sales for each Sub_Category, filtered by the two latest years
    SELECT
        t1.Sub_Category,
        EXTRACT(YEAR FROM t1.Order_Date) AS sales_year,
        SUM(t1.Sales) AS yearly_sales
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean` AS t1
    CROSS JOIN
        MaxYears AS m
    WHERE
        EXTRACT(YEAR FROM t1.Order_Date) IN (m.Y1, m.Y2)
    GROUP BY
        1, 2
)
SELECT
    y.Sub_Category,
    -- Pivot the sales data using the dynamic years
    SUM(CASE WHEN y.sales_year = m.Y1 THEN y.yearly_sales ELSE 0 END) AS sales_y1,
    SUM(CASE WHEN y.sales_year = m.Y2 THEN y.yearly_sales ELSE 0 END) AS sales_y2,
    (
        SUM(CASE WHEN y.sales_year = m.Y2 THEN y.yearly_sales ELSE 0 END) -
        SUM(CASE WHEN y.sales_year = m.Y1 THEN y.yearly_sales ELSE 0 END)
    ) AS yoy_delta
FROM
    yr_sales AS y
CROSS JOIN
    MaxYears AS m -- Join MaxYears again to use Y1/Y2 in the final calculation/labeling
GROUP BY
    Sub_Category
ORDER BY
    yoy_delta DESC
LIMIT 5
```



## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

>ROW_NUMBER() is used here because the requirement specifies returning the single highest-revenue sub-category per region, ensuring only one row is returned even in the case of ties. RANK() would assign the same rank (rank 1) to multiple sub-categories if their sales totals were tied, resulting in more than one "highest-revenue" category being returned per region.

<br/>

**Reflection**
Chair and Phones are the highest revenue sub categories.
**SQL Query**
```
WITH subcat_sales AS (
    -- Calculate total sales for every Sub_Category within each Region
    SELECT
        Region,
        Sub_Category,
        SUM(Sales) AS total_sales
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
    GROUP BY
        Region,
        Sub_Category
),
ranked_subcat_sales AS (
    -- Rank Sub_Categories by sales within each Region
    SELECT
        Region,
        Sub_Category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
    FROM
        subcat_sales
)
-- Select only the top Sub_Category (rn = 1) for each Region
SELECT
    Region,
    Sub_Category,
    total_sales
FROM
    ranked_subcat_sales
WHERE
    rn = 1
ORDER BY
    Region
```

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

<br/>

**Reflection**
Revenue decreased from 2014-2015 but increased YoY from 15-17.

**SQL Query**
```
WITH yearly_sales AS (
    -- Step 1 & 2: Filter for Phones and aggregate sales by year
    SELECT
        EXTRACT(YEAR FROM Order_Date) AS sales_year,
        SUM(Sales) AS yearly_revenue
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
    WHERE
        Sub_Category = 'Phones'
    GROUP BY
        sales_year
),
lagged_sales AS (
    -- Step 3: Add previous year's revenue using LAG
    SELECT
        sales_year AS year,
        yearly_revenue,
        LAG(yearly_revenue) OVER (ORDER BY sales_year ASC) AS prev_revenue
    FROM
        yearly_sales
)
-- Step 4: Compute YoY percentage growth, guarding against NULLs and zero division
SELECT
    year,
    yearly_revenue,
    prev_revenue,
    -- Handle the first year (prev_revenue IS NULL) and potential divide-by-zero (prev_revenue = 0)
    CASE
        WHEN prev_revenue IS NULL THEN NULL
        WHEN prev_revenue = 0 THEN NULL -- Cannot calculate growth if previous revenue was zero
        ELSE 100.0 * (yearly_revenue - prev_revenue) / prev_revenue
    END AS yoy_pct
FROM
    lagged_sales
ORDER BY
    year ASC
```

### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

<br/>

**Reflection**
Nov 2017 and Dec 2016 had the highest moving average of monthly revenue.
**SQL Query**
```
WITH monthly_sales AS (
    -- Filter for Corporate segment and aggregate monthly sales
    SELECT
        DATE_TRUNC(Order_Date, MONTH) AS month,
        SUM(Sales) AS monthly_revenue
    FROM
        `mgmt-467-nh.lab1_foundation.superstore_clean`
    WHERE
        Segment = 'Corporate'
    GROUP BY
        month
)
-- Compute the 3-month moving average
SELECT
    month,
    monthly_revenue,
    AVG(monthly_revenue) OVER (
        ORDER BY month ASC
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS ma_3
FROM
    monthly_sales
ORDER BY
    month ASC
```

## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

<br/>

**Reflection**
This prompt helped diagnose the errors from the query prompts for years outside of the dataset. Using VertexAI Studio I was able to identify the issue and correct the query.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

<br/>

**Reflection**

For the query from E3:

1. Table Partitioning (Most Effective):
> Action: If the source table (superstore_clean) were partitioned by the Order_Date column (e.g., by day or month), the WHERE Segment = 'Corporate' filter could be combined with a date range filter (if only recent data is needed) or BigQuery's storage optimization automatically skips partitions irrelevant to the segment if the table is clustered/partitioned appropriately. However, the primary benefit here is date range filtering. If the table is partitioned by Order_Date, adding a simple time filter like WHERE Order_Date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) would drastically limit the partitions read, saving significant bytes.

2. Column Pruning (Selecting Only Necessary Columns):
> Action: The current query reads the entire superstore_clean table just to access Order_Date, Sales, and Segment. If the table contained many other columns (e.g., Customer_Name, Product_ID, Notes), performance is hurt by scanning these unnecessary fields. The query should be explicitly changed to SELECT Order_Date, Sales, Segment FROM ... in the monthly_sales CTE to enforce column pruning and reduce the scanned data volume.

3. Table Clustering (Optimizing Filter):
> Action: Ensure the superstore_clean table is clustered by the Segment column. BigQuery uses clustering to colocate data with similar values. Since the query immediately filters on WHERE Segment = 'Corporate', clustering the table on Segment would allow BigQuery to quickly skip reading blocks of data that only contain other segments (Consumer, Home Office), significantly reducing the I/O needed to find the relevant rows.


## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

<br/>

**Reflection:**

To compare the outcomes, analyze the total_profit column across the groupings provided by the two new queries. If Query 1 shows that profit for 'Tables' is positive in certain Region/Year combinations, it falsifies the idea that 'Tables' is universally unprofitable. Similarly, if Query 2 demonstrates that the bulk of the loss comes exclusively from orders shipped via the most expensive Ship_Mode, it strongly suggests the negative profit is primarily driven by logistics costs, not product discounting, thereby nuancing the original conclusion.

**SQL Query:**
1. ```SELECT
    EXTRACT(YEAR FROM Order_Date) AS sales_year,
    Region,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    AVG(Discount) AS average_discount
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
WHERE
    Sub_Category = 'Tables'
GROUP BY
    1, 2
ORDER BY
    sales_year DESC, total_profit ASC


2. ```SELECT
    Ship_Mode,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    AVG(Discount) AS average_discount,
    COUNT(Ship_Mode) AS order_count
FROM
    `mgmt-467-nh.lab1_foundation.superstore_clean`
WHERE
    Sub_Category = 'Tables'
GROUP BY
    Ship_Mode
ORDER BY
    total_profit ASC


## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

<br/>

**Reflection**

The 'Tables' sub-category is currently unprofitable, representing a significant drain on overall business performance. This financial loss is likely driven by a combination of high average discounts and elevated shipping costs, as indicated by regional and shipping mode analysis. We recommend the Product and Logistics teams immediately review the discounting and shipping policies for the 'Tables' category to identify areas for cost reduction. To measure the impact of these changes, we will monitor the total_profit and average_discount for this sub-category on a monthly basis.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```



---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---